In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
np.set_printoptions(precision = 3)

# Preparation for statistics modelling

In [9]:
pu_2019 = pd.read_csv('../data/curated/pu_2019.csv')
do_2019 = pd.read_csv('../data/curated/pu_2020.csv')
pu_2020 = pd.read_csv('../data/curated/pu_2020.csv')
do_2020 = pd.read_csv('../data/curated/do_2020.csv')
all_data = [pu_2019, do_2019, pu_2020, do_2020]

# Drop duplicate rows
weather_2019 = pu_2019.drop_duplicates(subset = ['month'])
weather_2020 = pu_2020.drop_duplicates(subset = ['month'])

for data in all_data:
    data.drop_duplicates(subset = ['LocationID'], inplace = True)
    data['ln_trip_count_total'] = np.log(data['trip_count_total'])
    data['ln_Price_per_square_feet'] = np.log(data['Price_per_square_feet'])
    data['ln_Population_By_LocationID'] = np.log(data['Population_By_LocationID'])
    data['ln_Density_per_hectare'] = np.log(data['Density_per_hectare'])

In [10]:
def print_model_performance(true, pred, model):
    """
    print the summary, anova table and rmse of a given linear model
    """
    print(model.summary())
    print("=" * 78)
    anova_table = sm.stats.anova_lm(model, robust = "hc2")
    print("ANOVA TABLE")
    print(anova_table)
    print("=" * 78)
    mask = ~np.isnan(true) & ~np.isnan(pred)
    true = true[mask]
    pred = pred[mask]
    print('RMSE = {:<.4}'.format(np.sqrt(mean_squared_error(true, pred))))
    
    
def stepwise_selection(data, response, all_covariates, __covariates=None, __min_aic=np.inf, counter=1):
    """
    print the model attributes with lowest aic
    """
    print(f"Stage {counter}")
    before_aic = __min_aic
    min_aic = __min_aic
    if __min_aic == np.inf:
        __covariates = all_covariates
    # Remove one covariate and check the aic 
    for var in __covariates:
        new_covariates = [x for x in __covariates if x != var]
        formula = response + '~' + '+'.join(new_covariates)
        curr_model = ols(formula = formula, data = data).fit()
        print(f"{new_covariates}: {curr_model.aic}")
        if curr_model.aic < min_aic:
            min_aic = curr_model.aic
            best_model = curr_model
    # Add one covariate and check the aic
    for var in all_covariates:
        if var in __covariates:
            continue
        new_covariates = list(__covariates) + [var, ]
        formula = response + '~' + '+'.join(new_covariates)
        curr_model = ols(formula = formula, data = data).fit()
        print(f"{new_covariates}: {curr_model.aic}")
        if curr_model.aic < min_aic:
            min_aic = curr_model.aic
            best_model = curr_model
    if before_aic == min_aic:
        print(f"Best of Final: {list(__covariates)} with aic = {before_aic}\n")
    else:
        __covariates = best_model.params.index[1:]
        print(f"Best of Stage {counter}: {list(__covariates)} with aic = {best_model.aic}")
        return stepwise_selection(data, response, all_covariates, __covariates, min_aic, counter=counter+1)

# Modelling

## How does weather impact the number of trips?

In [11]:
model0 = ols(formula = "trip_count_in_month ~ 1",
             data = weather_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model0.predict(weather_2020)
print_model_performance(weather_2020['trip_count_in_month'], trip_count_pred, model0)

                             OLS Regression Results                            
Dep. Variable:     trip_count_in_month   R-squared:                      -0.000
Model:                             OLS   Adj. R-squared:                 -0.000
Method:                  Least Squares   F-statistic:                       nan
Date:                 Mon, 02 Oct 2023   Prob (F-statistic):                nan
Time:                         15:57:49   Log-Likelihood:                -176.60
No. Observations:                   12   AIC:                             355.2
Df Residuals:                       11   BIC:                             355.7
Df Model:                            0                                         
Covariance Type:                   HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.453e+06    1.8e+05     35.90

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [12]:
stepwise_selection(weather_2019, 
                   'trip_count_in_month', 
                   ['feelslike', 'precip', 'windspeed', 'visibility', 'snow', 'snowdepth']
                   )
model1 = ols(formula = "trip_count_in_month ~ windspeed + visibility + snow + snowdepth",
             data = weather_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model1.predict(weather_2020)
print_model_performance(weather_2020['trip_count_in_month'], trip_count_pred, model1)

Stage 1
['precip', 'windspeed', 'visibility', 'snow', 'snowdepth']: 329.7807298377206
['feelslike', 'windspeed', 'visibility', 'snow', 'snowdepth']: 331.5688226895277
['feelslike', 'precip', 'visibility', 'snow', 'snowdepth']: 342.4714215089548
['feelslike', 'precip', 'windspeed', 'snow', 'snowdepth']: 337.48268110804077
['feelslike', 'precip', 'windspeed', 'visibility', 'snowdepth']: 350.3016083464621
['feelslike', 'precip', 'windspeed', 'visibility', 'snow']: 353.3592973962748
Best of Stage 1: ['precip', 'windspeed', 'visibility', 'snow', 'snowdepth'] with aic = 329.7807298377206
Stage 2
['windspeed', 'visibility', 'snow', 'snowdepth']: 329.6553835187358
['precip', 'visibility', 'snow', 'snowdepth']: 354.55438077889653
['precip', 'windspeed', 'snow', 'snowdepth']: 335.48922132414583


['precip', 'windspeed', 'visibility', 'snowdepth']: 349.225415878455
['precip', 'windspeed', 'visibility', 'snow']: 351.4536714484183
['precip', 'windspeed', 'visibility', 'snow', 'snowdepth', 'feelslike']: 331.77441279540744
Best of Stage 2: ['windspeed', 'visibility', 'snow', 'snowdepth'] with aic = 329.6553835187358
Stage 3
['visibility', 'snow', 'snowdepth']: 355.0132032323735
['windspeed', 'snow', 'snowdepth']: 336.8205955564831
['windspeed', 'visibility', 'snowdepth']: 347.234416087303
['windspeed', 'visibility', 'snow']: 349.46656801965867
['windspeed', 'visibility', 'snow', 'snowdepth', 'feelslike']: 331.5688226895277
['windspeed', 'visibility', 'snow', 'snowdepth', 'precip']: 329.7807298377206
Best of Final: ['windspeed', 'visibility', 'snow', 'snowdepth'] with aic = 329.6553835187358

                             OLS Regression Results                            
Dep. Variable:     trip_count_in_month   R-squared:                       0.939
Model:                            

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [13]:
# Lasso regression
y = ['trip_count_in_month']
X = ['feelslike', 'precip', 'windspeed', 'visibility', 'snow', 'snowdepth']
reg = LassoCV(cv = 5, 
              normalize = True, 
              random_state = 0).fit(weather_2019[X], np.array(weather_2019[y]).ravel())


r2 = reg.score(weather_2019[X], weather_2019[y])
trip_count_pred = reg.predict(weather_2020[X])
rmse = np.sqrt(mean_squared_error(weather_2020[y], trip_count_pred))


formula = 'trip_count_in_month = ' + str(reg.intercept_)[:10] + ' + ' \
            + ' + '.join([str(coef)[:10] + ' * ' + attr for coef, attr in zip(reg.coef_, X)])


print(formula)
print('a = {:<.4}'.format(reg.alpha_))
print('R2 = {:<.4}'.format(r2))
print('RMSE = {:<.4}'.format(rmse))

trip_count_in_month = 10316969.7 + -777.91157 * feelslike + -76968.009 * precip + 120234.948 * windspeed + -424543.97 * visibility + -3081987.8 * snow + 1632808.86 * snowdepth
a = 133.6
R2 = 0.9478
RMSE = 1.622e+06


## How do populations and property prices impact the pick-up number in a zone?

In [14]:
stepwise_selection(pu_2019, 
                   'ln_trip_count_total', 
                   ['ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']
                   )

model2 = ols(formula = "ln_trip_count_total ~ ln_Price_per_square_feet + ln_Population_By_LocationID \
                        + ln_Density_per_hectare",
             data = pu_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model2.predict(pu_2020)
print_model_performance(pu_2020['ln_trip_count_total'], trip_count_pred, model2)

Stage 1
['ln_Population_By_LocationID', 'ln_Density_per_hectare']: 1191.6306456449654
['ln_Price_per_square_feet', 'ln_Density_per_hectare']: 1202.2730215821123
['ln_Price_per_square_feet', 'ln_Population_By_LocationID']: 1232.963912940024
Best of Stage 1: ['ln_Population_By_LocationID', 'ln_Density_per_hectare'] with aic = 1191.6306456449654
Stage 2
['ln_Density_per_hectare']: 1276.530487533489
['ln_Population_By_LocationID']: 1309.0211083255085
['ln_Population_By_LocationID', 'ln_Density_per_hectare', 'ln_Price_per_square_feet']: 1140.4996405237382
Best of Stage 2: ['ln_Population_By_LocationID', 'ln_Density_per_hectare', 'ln_Price_per_square_feet'] with aic = 1140.4996405237382
Stage 3
['ln_Density_per_hectare', 'ln_Price_per_square_feet']: 1202.2730215821123
['ln_Population_By_LocationID', 'ln_Price_per_square_feet']: 1232.963912940024
['ln_Population_By_LocationID', 'ln_Density_per_hectare']: 1191.6306456449654
Best of Final: ['ln_Population_By_LocationID', 'ln_Density_per_hectare

## How do populations and property prices impact the drop-off number in a zone?

In [15]:
stepwise_selection(do_2019, 
                   'ln_trip_count_total', 
                   [ 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']
                   )

model3 = ols(formula = "ln_trip_count_total ~ ln_Price_per_square_feet + ln_Population_By_LocationID \
                        + ln_Density_per_hectare",
             data = do_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model3.predict(do_2020)
print_model_performance(do_2020['ln_trip_count_total'], trip_count_pred, model3)

Stage 1
['ln_Population_By_LocationID', 'ln_Density_per_hectare']: 1216.133662201194
['ln_Price_per_square_feet', 'ln_Density_per_hectare']: 1298.305493057976
['ln_Price_per_square_feet', 'ln_Population_By_LocationID']: 1334.5240422916254
Best of Stage 1: ['ln_Population_By_LocationID', 'ln_Density_per_hectare'] with aic = 1216.133662201194
Stage 2
['ln_Density_per_hectare']: 1304.0949285854415
['ln_Population_By_LocationID']: 1338.3189587327447
['ln_Population_By_LocationID', 'ln_Density_per_hectare', 'ln_Price_per_square_feet']: 1208.650326344783
Best of Stage 2: ['ln_Population_By_LocationID', 'ln_Density_per_hectare', 'ln_Price_per_square_feet'] with aic = 1208.650326344783
Stage 3
['ln_Density_per_hectare', 'ln_Price_per_square_feet']: 1298.305493057976
['ln_Population_By_LocationID', 'ln_Price_per_square_feet']: 1334.5240422916254
['ln_Population_By_LocationID', 'ln_Density_per_hectare']: 1216.133662201194
Best of Final: ['ln_Population_By_LocationID', 'ln_Density_per_hectare', '